<a href="https://colab.research.google.com/github/PJINU/AIFFEL-GOING_DEEPER/blob/main/USA_86days/1_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# custom datset 만들기(easy.ver)

import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset1(Dataset):
    def __init__(self,t):
        self.t = t

    def __len__(self):
        return self.t

    def __getitem__(self,idx):
        return torch.LongTensor([idx])


if __name__ == '__main__':
    dataset = MyDataset1(t=5)
    print(len(dataset))
    it = iter(dataset)

    for i in range(10):
        print(i, next(it))


5
0 tensor([0])
1 tensor([1])
2 tensor([2])
3 tensor([3])
4 tensor([4])
5 tensor([5])
6 tensor([6])
7 tensor([7])
8 tensor([8])
9 tensor([9])


### DataLoader의 batch_size와 Dataset의 index 사이의 관계
- batch_size만큼 dataset에서 데이터를 가져온다. batch_size만큼 임의의 인덱스를 뽑아주면 그 인덱스 번호에 맞는 데이터를 가져온다. shuffle을 True로 설정하면 데이터가 섞여서 가져오지만 False로 설정하면 데이터를 순서대로 가져온다.
    - ex)  
        data = [1,2,3,4,5,6,7,8,9,10]  
        batch_size = 3 -> 임의의 번호 3개(index)를 선정 => (0,1,4), (3,5,2), (9,7,8), (6)  
        data에서 첫번째로 (0,1,4)번에 해당하는 데이터 (1,2,5)을 가져옴  
        두번째로 (3,5,2)번에 해당하는 데이터 (4,6,3)을 가져옴  
        세번째로 (9,7,8)번에 해당하는 데이터 (10,8,9)을 가져옴  
        마지막으로 (6)번에 해당하는 데이터 (7)을 가져옴

- DataLoader가 Dataset의 len을 먼저 호출하여 전체 데이터 개수를 확인 -> 전체 데이터 개수를 확인후 그 안에서 인덱스 번호를 임의로 선정

- batch에 들어가는 데이터의 길이는 동일해야 한다. 만약 동일하지 않은 경우는 어떻게 해야 할까?? ex) 문장
    - 텍스트 데이터로 짜보기

In [38]:
# custom datset 만들기2(easy.ver)

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# input_data = np.array([1,2,3,4,5,6,7,8,9,10,11,12])
input_data = np.arange(12).reshape(3,4)


class MyDataset1(Dataset):
    def __init__(self, data):
        self.data = data
        self.length = len(data)

    def __len__(self):
        return self.length

    def __getitem__(self, idx): # 한 개의 데이터에 대해서 적용할 
        x = np.array([idx])
        y = self.data[idx]
        return x, y


if __name__ == '__main__':
    dataset = MyDataset1(input_data)
    dataloader = DataLoader(dataset = dataset, batch_size=5, shuffle = True)
    for loop in range(2):
        print(loop)
        for batch in dataloader:
            data_index, output_data = batch
            print(f'index : {data_index}')
            print(f'data : {output_data}')
            print('---')

0
index : tensor([[1],
        [2],
        [0]])
data : tensor([[ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [ 0,  1,  2,  3]])
---
1
index : tensor([[2],
        [1],
        [0]])
data : tensor([[ 8,  9, 10, 11],
        [ 4,  5,  6,  7],
        [ 0,  1,  2,  3]])
---


In [4]:
# custom datset 만들기2(easy.ver)

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# input_data = np.array([1,2,3,4,5,6,7,8,9,10,11,12])
# input_data = np.array([[1,2],[3,4,5],[6,7]])
# input_data = np.arange(12).reshape(3,4)
input_data = np.random.randint(0, 3000, (10,3,10,10))


class MyDataset1(Dataset):
    def __init__(self, data):
        self.data = data
        self.length = len(data)

    def __len__(self):
        return self.length

    def __getitem__(self, idx): # 한 개의 데이터에 대해서 적용할 
        x = np.array([idx])
        y = self.data[idx]
        return x, y


if __name__ == '__main__':
    dataset = MyDataset1(input_data)
    dataloader = DataLoader(dataset = dataset, batch_size=5, shuffle = True)
    for loop in range(2):
        print(loop)
        for batch in dataloader:
            data_index, output_data = batch
            print(f'index : {data_index.shape}')
            print(f'data : {output_data.shape}')
            # print('---')

0
index : torch.Size([5, 1])
data : torch.Size([5, 3, 10, 10])
index : torch.Size([5, 1])
data : torch.Size([5, 3, 10, 10])
1
index : torch.Size([5, 1])
data : torch.Size([5, 3, 10, 10])
index : torch.Size([5, 1])
data : torch.Size([5, 3, 10, 10])


### 데이터의 길이가 다른 경우 

In [5]:
sentences = [
            'I like apple',
            'I dont like apple',
            'My name is Tony Stark'
]



In [6]:
# 토큰화

tokenized = []

for sentence in sentences:
    tokenized.append(sentence.split())

print(tokenized)

[['I', 'like', 'apple'], ['I', 'dont', 'like', 'apple'], ['My', 'name', 'is', 'Tony', 'Stark']]


In [7]:
!pip install nltk

In [9]:
# 빈도수별로 단어장 형성

from nltk import FreqDist

vocab = FreqDist(np.hstack(tokenized))
print(len(vocab))

9


In [14]:
# 단어별 인덱스 설정

word_to_index = {}

for index, word in enumerate(vocab):
    word_to_index[word] = index + 2
word_to_index['pad'] = 1
word_to_index['unk'] = 0

word_to_index


{'I': 2,
 'My': 6,
 'Stark': 10,
 'Tony': 9,
 'apple': 4,
 'dont': 5,
 'is': 8,
 'like': 3,
 'name': 7,
 'pad': 1,
 'unk': 0}

In [15]:
# 단어를 인덱스로 바꿔주기(인코딩)

encoded = []
for line in tokenized:
    temp = []
    for w in line:
        try:
            temp.append(word_to_index[w])
        except:
            temp.append(word_to_index['unk'])

    encoded.append(temp)

encoded

[[2, 3, 4], [2, 5, 3, 4], [6, 7, 8, 9, 10]]

In [17]:
max_len = max(len(l) for l in encoded)
max_len

5

In [20]:
for line in encoded:
    if len(line) < max_len:
        line += [word_to_index['pad']] * (max_len - len(line))

encoded

[[2, 3, 4, 1, 1], [2, 5, 3, 4, 1], [6, 7, 8, 9, 10]]